In [ ]:
import openforcefield as off
from rdkit import Chem
from rdkit.Chem import AllChem
from simtk import openmm, unit
from simtk.openmm import app
from openforcefield.topology import Topology
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField
from rdkit.Chem.Draw import IPythonConsole

from simtk.openmm.openmm import Platform


In [ ]:
import openmmtools
pts = openmmtools.utils.get_available_platforms()

for plat in pts:
    print(plat.getName())
    
openmmtools.utils.get_fastest_platform().getName()

In [ ]:
ff = ForceField('test_forcefields/smirnoff99Frosst.offxml')

In [ ]:
%time rdmol = Chem.MolFromSmiles('CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC')
%time rdmol = Chem.AddHs(rdmol)
%time AllChem.EmbedMolecule(rdmol)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(rdmol)
%time ofmol = Molecule.from_rdkit(rdmol)

# ofmol.generate_conformers(n_conformers=10)
%time topology = ofmol.to_topology()

%time org_system = ff.create_openmm_system(topology)
# pos = ofmol.conformers[0]



In [ ]:
pos = ofmol.conformers[0]

In [ ]:
def get_energy(system, positions, platform):
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy

rdmol = Chem.MolFromSmiles('CCCCCCCCCCCCCCCC')
rdmol = Chem.AddHs(rdmol)
AllChem.EmbedMolecule(rdmol)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(rdmol)
ofmol = Molecule.from_rdkit(rdmol)

# ofmol.generate_conformers(n_conformers=10)
topology = ofmol.to_topology()

org_system = ff.create_openmm_system(topology)
pos = ofmol.conformers[0]

In [ ]:
for plat in openmmtools.utils.get_available_platforms():
    print(plat.getName())
    %timeit get_energy(org_system, pos, plat)

In [ ]:
from simtk.openmm.openmm import Platform

new_system = ff.create_openmm_system(topology)
# new_energy = get_energy(new_system, pos)
from sys import stdout
def minimizeOpenMM(Topology, System, Positions, platform):
    
    platform = Platform.getPlatformByName('CPU')
#     properties = {'DeviceIndex': '0', 'Precision': 'double'}
    integrator = openmm.LangevinIntegrator(
                                        300.0 * unit.kelvin,
                                        1.0 / unit.picosecond,
                                        2.0 * unit.femtosecond)
                                        #.002 * unit.picoseconds)
    simulation = app.Simulation(Topology, System, integrator, platform)
    simulation.context.setPositions(Positions)
    simulation.minimizeEnergy(tolerance=5.0E-9, maxIterations=2000)
#     state =  simulation.context.getState(getPositions=True, getEnergy=True)
#     positions =state.getPositions(asNumpy=True)
#     energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
     
#     simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
#     simulation.step(30000)
#     print(energy)
#     positions = positions / unit.angstroms
#     coordlist = list()
#     for atom_coords in positions:
#         coordlist += [i for i in atom_coords]
#     return coordlist, positions
    return None, None

for plat in openmmtools.utils.get_available_platforms():
    print(plat.getName())
    ofmol.generate_conformers(n_conformers=1)
    topology = ofmol.to_topology()

    org_system = ff.create_openmm_system(topology)
    pos = ofmol.conformers[0]
    %time _, _=minimizeOpenMM(topology, org_system, pos, plat)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import py3Dmol
from simtk.openmm import openmm

from openforcefield.topology import Molecule

# Define the keyword arguments to feed to ForceField
from simtk import unit
from simtk.openmm import app

In [42]:
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SDWriter
from utils import *
# molOrig = Chem.MolFromSmiles("COc1cc(C2Oc3c(OC)cc(C=CCO)cc3C2CO)ccc1[O]")
molOrig = Chem.MolFromSmiles('COc1cc(C(O)C(CO)Oc2c(OC)cc(C(O)C(CO)Oc3ccc(C4Oc5c(OC)cc(C6Oc7c(OC)cc(C8Oc9c(OC)cc(C=CCO)cc9C8CO)cc7C6CO)cc5C4CO)cc3OC)cc2-c2cc(C3OCC4C(c5ccc(O)c(OC)c5)OCC34)cc(OC)c2O)ccc1O')
molOrig = Chem.AddHs(molOrig)
res = Chem.AllChem.EmbedMultipleConfs(molOrig, numConfs=2, numThreads=-1)

%time Chem.AllChem.MMFFOptimizeMoleculeConfs(molOrig, numThreads=-1)

confgen = ConformerGeneratorCustom(max_conformers=1,
                             rmsd_threshold=None,
                             force_field='mmff',
                             pool_multiplier=1)

%time confgen.get_conformer_energies(molOrig)

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 686 ms
CPU times: user 27.7 ms, sys: 0 ns, total: 27.7 ms
Wall time: 27.7 ms


array([472.45003705, 485.17011093])

In [43]:
import numpy as np
AllChem.ComputeGasteigerCharges(molOrig)

pcs = []

for i in range(molOrig.GetNumAtoms()):
    pcs.append(molOrig.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge'))
    
charges = np.array(pcs)

In [44]:
# AllChem.EmbedMolecule(molOrig)
# Chem.rdmolops.AssignAtomChiralTagsFromStructure(molOrig)
from openforcefield.topology import Molecule
ofmol = Molecule.from_rdkit(molOrig, allow_undefined_stereo=True)
from simtk import unit

# ofmol = Molecule.from_smiles('COc1cc(C2Oc3c(OC)cc(C=CCO)cc3C2CO)ccc1[O]', allow_undefined_stereo=True)
# ofmol = Molecule.from_smiles(ofmol.to_smiles(), allow_undefined_stereo=True)

#generate_unique_atom_names(ofmol)
# %time ofmol.generate_conformers(n_conformers=1)

pos = ofmol.conformers[0]
pos1 = ofmol.conformers[1]
ofmol.name = 'molecule'

ofmol.partial_charges += charges * unit.elementary_charge

ofmol.partial_charges

Quantity(value=array([ 0.07771734, -0.49286616,  0.1603491 , -0.0101442 , -0.01182129,
        0.11823621, -0.38442422,  0.15176771,  0.08302896, -0.39240245,
       -0.48038315,  0.16934092,  0.16128531, -0.49283231,  0.07771814,
       -0.01008842, -0.01119942,  0.11825714, -0.38442384,  0.15173394,
        0.08302835, -0.39240246, -0.48104669,  0.16139582, -0.01563976,
       -0.05134959, -0.00303046,  0.13324475, -0.48079387,  0.16536303,
        0.16098846, -0.49284448,  0.07771784, -0.00912805, -0.00274096,
        0.13325486, -0.48079368,  0.16536303,  0.16098846, -0.49284448,
        0.07771784, -0.00912805, -0.00274095,  0.13325505, -0.48079331,
        0.16538187,  0.16117101, -0.49283474,  0.0777181 , -0.00881508,
       -0.02164807, -0.0565704 , -0.05867551,  0.0615174 , -0.39229545,
       -0.04726519,  0.00808551,  0.05220159,  0.05398895, -0.39557623,
       -0.04758306,  0.00790345,  0.05219233,  0.05398876, -0.39557623,
       -0.04758306,  0.00790345,  0.05219215,  0.

In [45]:
from simtk.openmm import app
from simtk import unit
forcefield_kwargs = { 'constraints' : app.HBonds, 'rigidWater' : True, 'removeCMMotion' : False, 'hydrogenMass' : 4*unit.amu }
# Initialize a SystemGenerator using GAFF
from openmmforcefields.generators import SystemGenerator
system_generator = SystemGenerator(small_molecule_forcefield='gaff-2.11', molecules=[ofmol], forcefield_kwargs=forcefield_kwargs, cache='db.json')
# Create an OpenMM System from an OpenMM Topology object
%time system = system_generator.create_system(ofmol.to_topology().to_openmm())
# takes nearly 2 minutes wall time on first go, have to ask why

Checking against [H][O][C]1=[C]([H])[C]([H])=[C]([C]([H])([O][H])[C]([H])([O][C]2=[C]([O][C]([H])([H])[H])[C]([H])=[C]([C]([H])([O][H])[C]([H])([O][C]3=[C]([H])[C]([H])=[C]([C]4([H])[O][C]5=[C]([O][C]([H])([H])[H])[C]([H])=[C]([C]6([H])[O][C]7=[C]([O][C]([H])([H])[H])[C]([H])=[C]([C]8([H])[O][C]9=[C]([O][C]([H])([H])[H])[C]([H])=[C]([C]([H])=[C]([H])[C]([H])([H])[O][H])[C]([H])=[C]9[C]8([H])[C]([H])([H])[O][H])[C]([H])=[C]7[C]6([H])[C]([H])([H])[O][H])[C]([H])=[C]5[C]4([H])[C]([H])([H])[O][H])[C]([H])=[C]3[O][C]([H])([H])[H])[C]([H])([H])[O][H])[C]([H])=[C]2[C]2=[C]([H])[C]([C]3([H])[O][C]([H])([H])[C]4([H])[C]([H])([C]5=[C]([H])[C]([H])=[C]([O][H])[C]([O][C]([H])([H])[H])=[C]5[H])[O][C]([H])([H])[C]34[H])=[C]([H])[C]([O][C]([H])([H])[H])=[C]2[O][H])[C]([H])([H])[O][H])[C]([H])=[C]1[O][C]([H])([H])[H]
CPU times: user 262 ms, sys: 0 ns, total: 262 ms
Wall time: 262 ms


In [46]:
def get_energy(system, positions, platform):
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy


import openmmtools
pts = openmmtools.utils.get_available_platforms()

for plat in pts:
    print(plat.getName())
    %time energy = get_energy(system, pos, plat)
    print(energy)

Reference
CPU times: user 5.84 ms, sys: 0 ns, total: 5.84 ms
Wall time: 4.86 ms
340.2725773267236 kcal/mol
CPU
CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 8.2 ms
340.27257006872236 kcal/mol
CUDA
CPU times: user 163 ms, sys: 35.8 ms, total: 199 ms
Wall time: 245 ms
340.2725423955097 kcal/mol
OpenCL
CPU times: user 45 ms, sys: 130 ms, total: 175 ms
Wall time: 262 ms
340.27257157101457 kcal/mol


In [39]:
energy = get_energy(system, pos, plat)
energy1 = get_energy(system, pos1, plat)

print(energy, energy1)

360.0125431330664 kcal/mol 347.5445953427491 kcal/mol


In [57]:
from simtk.openmm.openmm import Platform


def minimizeOpenMM(Topology, System, Positions, platform):
    
    platform = Platform.getPlatformByName('CPU')
#     properties = {'DeviceIndex': '0', 'Precision': 'double'}
    integrator = openmm.LangevinIntegrator(
                                        500.0 * unit.kelvin,
                                        1.0 / unit.picosecond,
                                        2.0 * unit.femtosecond)
                                        #.002 * unit.picoseconds)
    simulation = app.Simulation(Topology, System, integrator, platform)
    simulation.context.setPositions(Positions)
    simulation.minimizeEnergy(tolerance=5.0E-1, maxIterations=2000)
#     state =  simulation.context.getState(getPositions=True, getEnergy=True)
#     positions =state.getPositions(asNumpy=True)
#     energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
     
#     simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
#     simulation.step(30000)
#     print(energy)
#     positions = positions / unit.angstroms
#     coordlist = list()
#     for atom_coords in positions:
#         coordlist += [i for i in atom_coords]
#     return coordlist, positions
    return None, None

for plat in openmmtools.utils.get_available_platforms():
    print(plat.getName())
    %time _, _=minimizeOpenMM(ofmol.to_topology(), system, pos, plat)

Reference
CPU times: user 4.71 s, sys: 11 s, total: 15.7 s
Wall time: 9.18 s
CPU
CPU times: user 4.46 s, sys: 10.9 s, total: 15.4 s
Wall time: 8.98 s
CUDA
CPU times: user 4.02 s, sys: 9.44 s, total: 13.5 s
Wall time: 7.86 s
OpenCL
CPU times: user 3.9 s, sys: 9.78 s, total: 13.7 s
Wall time: 8.01 s
